## Spark Exercises

In [1]:
import pandas as pd
import pyspark
from pydataset import data
from vega_datasets import data as v_data
import pyspark.sql.functions as F
from pyspark.sql.functions import asc, desc

### I. Create a spark data frame that contains your favorite programming languages.

1) The name of the column should be language

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
df = spark.createDataFrame(pd.DataFrame(['python', 'javascript','matlab','sql','javascript','r'], columns=['language']))
df

DataFrame[language: string]

In [4]:
df.show()

+----------+
|  language|
+----------+
|    python|
|javascript|
|    matlab|
|       sql|
|javascript|
|         r|
+----------+



2) View the schema of the dataframe

In [5]:
df.printSchema()

root
 |-- language: string (nullable = true)



3) Output the shape of the dataframe

In [6]:
df.count(), len(df.columns)

(6, 1)

4) Show the first 5 records in the dataframe

In [7]:
df.show(5)

+----------+
|  language|
+----------+
|    python|
|javascript|
|    matlab|
|       sql|
|javascript|
+----------+
only showing top 5 rows



### II. Load the mpg dataset as a spark dataframe.

In [8]:
mpg = spark.createDataFrame(data('mpg'))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



1) Create 1 column of output that contains a message like the one below:

> The 1999 audi a4 has a 4 cylinder engine.

For each vehicle.

In [9]:
col = F.concat(F.lit('The '), 
                    mpg.year, F.lit(' '), 
                    mpg.manufacturer, 
                    F.lit(' '), 
                    mpg.model, 
                    F.lit(' has a '), 
                    mpg.cyl,
                    F.lit(' cylinder engine.')).alias('description')
mpg.select(col).show(5,truncate=False)

+-----------------------------------------+
|description                              |
+-----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 6 cylinder engine.|
+-----------------------------------------+
only showing top 5 rows



2) Transform the trans column so that it only contains either manual or auto.

In [10]:
simple_trans = F.regexp_extract(mpg.trans, 
                                r'(^\w+?)\(.*\)', 
                                1).alias('simple_trans')
mpg.select(simple_trans).show(5)

+------------+
|simple_trans|
+------------+
|        auto|
|      manual|
|      manual|
|        auto|
|        auto|
+------------+
only showing top 5 rows



### III. Load the tips dataset as a spark dataframe.

In [11]:
tips = spark.createDataFrame(data('tips'))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



1) What percentage of observations are smokers?

In [12]:
smoker_or_no = F.when(tips.smoker == 'Yes', 1).otherwise(0)
smoker_or_no

Column<b'CASE WHEN (smoker = Yes) THEN 1 ELSE 0 END'>

In [13]:
tips.select(F.mean(smoker_or_no)).show()

+-----------------------------------------------+
|avg(CASE WHEN (smoker = Yes) THEN 1 ELSE 0 END)|
+-----------------------------------------------+
|                            0.38114754098360654|
+-----------------------------------------------+



2) Create a column that contains the tip percentage

In [14]:
tip_percentage = F.round((tips.tip/tips.total_bill), 4).alias('tip_percentage')
tip_percentage

Column<b'round((tip / total_bill), 4) AS `tip_percentage`'>

In [15]:
tips.select('total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size', tip_percentage).show(10)

+----------+----+------+------+---+------+----+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage|
+----------+----+------+------+---+------+----+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        0.0594|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.1605|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.1666|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.1398|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        0.1468|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.1862|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.2281|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.1161|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|        0.1303|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|        0.2185|
+----------+----+------+------+---+------+----+--------------+
only showing top 10 rows



3) Calculate the average tip percentage for each combination of sex and smoker.

In [16]:
tips.select('sex', 'smoker', tip_percentage).groupBy('smoker').pivot('sex').agg(F.mean('tip_percentage')).show()

+------+-------------------+-------------------+
|smoker|             Female|               Male|
+------+-------------------+-------------------+
|    No| 0.1569111111111111| 0.1606659793814433|
|   Yes|0.18214545454545455|0.15276666666666663|
+------+-------------------+-------------------+



### IV. Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [17]:
weather = v_data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



Which month has the most rain, on average

1) Convert the temperatures to farenheight.

In [19]:
temp_to_f = lambda x: F.round((x * (9/5) + 32),2)
weather.select('temp_max', temp_to_f(weather.temp_max).alias('max_f_temp'), 'temp_min', temp_to_f(weather.temp_min).alias('min_f_temp')).show(10)

+--------+----------+--------+----------+
|temp_max|max_f_temp|temp_min|min_f_temp|
+--------+----------+--------+----------+
|    12.8|     55.04|     5.0|      41.0|
|    10.6|     51.08|     2.8|     37.04|
|    11.7|     53.06|     7.2|     44.96|
|    12.2|     53.96|     5.6|     42.08|
|     8.9|     48.02|     2.8|     37.04|
|     4.4|     39.92|     2.2|     35.96|
|     7.2|     44.96|     2.8|     37.04|
|    10.0|      50.0|     2.8|     37.04|
|     9.4|     48.92|     5.0|      41.0|
|     6.1|     42.98|     0.6|     33.08|
+--------+----------+--------+----------+
only showing top 10 rows



In [31]:
weather = (weather
           .withColumn('temp_max', (col('temp_max') * 9/5 + 32))
           .withColumn('temp_min', (col('temp_min') * 9/5 + 32)))

TypeError: 'Column' object is not callable

2) Which month has the most rain, on average?

In [20]:
month = F.regexp_extract(weather.date, r'(\w+?)\-(\w\w)\-\w\w', 2).alias('month')
weather.select(month, 'precipitation').where(weather.weather== 'rain').groupBy('month').mean().show()

+-----+-------------------+
|month| avg(precipitation)|
+-----+-------------------+
|   07| 1.8785714285714286|
|   11|               8.42|
|   01|  6.614285714285714|
|   09|0.22499999999999998|
|   05| 3.2624999999999997|
|   08|  6.433333333333334|
|   03|  4.921621621621622|
|   02| 3.1725000000000003|
|   06|  3.952631578947368|
|   10|              9.675|
|   12| 5.0260869565217385|
|   04|  3.429999999999999|
+-----+-------------------+



3) Which year was the windiest?

In [21]:
year = F.regexp_extract(weather.date, r'(\w+?)\-(\w\w)\-\w\w', 1).alias('year')
weather.select(year, 'wind').groupBy('year').mean().orderBy(F.desc('avg(wind)')).show()

+----+------------------+
|year|         avg(wind)|
+----+------------------+
|2012| 3.400819672131147|
|2014|3.3876712328767136|
|2015|  3.15972602739726|
|2013|3.0158904109589044|
+----+------------------+



4) What is the most frequent type of weather in January?

In [22]:
weather.select(month, 'weather').where(month == '01').groupBy('weather').count().orderBy(F.desc('count')).show()

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



4) What is the average high and low tempurature on sunny days in July in 2013 and 2014?

In [23]:
weather.select('temp_max','temp_min')\
       .where((month == '07')&
             ((year == '2013')|
              (year == '2014')))\
       .select(F.mean('temp_max'),F.mean('temp_min'))\
       .show()

+-----------------+------------------+
|    avg(temp_max)|     avg(temp_min)|
+-----------------+------------------+
|26.49677419354839|14.179032258064519|
+-----------------+------------------+



5) What percentage of days were rainy in q3 of 2015?

In [42]:
rainy_days =  F.when(weather.filter((year == '2015')&
                                   ((month == '07' )|
                                    (month == '08' )|
                                    (month == '09'))).weather == 'rain', 1)\
                            .otherwise(0)
weather.select(F.mean(rainy_days)).show()

+-------------------------------------------------+
|avg(CASE WHEN (weather = rain) THEN 1 ELSE 0 END)|
+-------------------------------------------------+
|                              0.17727583846680356|
+-------------------------------------------------+



In [32]:
rainy_days =  F.when(weather.filter((F.quarter('date') == 3)&(year == '2015')).weather == 'rain', 1).otherwise(0)
weather.select(F.mean(rainy_days)).show()

+-------------------------------------------------+
|avg(CASE WHEN (weather = rain) THEN 1 ELSE 0 END)|
+-------------------------------------------------+
|                              0.17727583846680356|
+-------------------------------------------------+



6) For each year, find what percentage of days it rained (had non-zero precipitation)

In [25]:
rained_or_no = F.when(weather.precipitation != '0', 1).otherwise(0).alias('rained')
weather.select('*', rained_or_no).groupBy(year).agg(F.mean('rained')).show()

+----+-------------------+
|year|        avg(rained)|
+----+-------------------+
|2012|0.48360655737704916|
|2014|  0.410958904109589|
|2013|0.41643835616438357|
|2015|0.39452054794520547|
+----+-------------------+

